Importing the required libraries

In [4]:
import numpy as np
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM

from keras.optimizers import RMSprop

from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
import random
import sys

Using TensorFlow backend.


Loading the data 

In [5]:
with open('034.txt', 'r') as file:
    text = file.read().lower()
    
    
print(text)    
    
    

a certain tailor had a son, who happened to be small, and
no bigger than a thumb, and on this account he was always
called thumbling.  he had, however, some courage in him, and
said to his father, father, i must and will go out into the
world.  that's right, my son, said the old man, and took a
long darning-needle and made a knob of sealing-wax on it at the
handle, and there is a sword for you to take with you on the way.
then the little tailor wanted to have one more meal with them,
and hopped into the kitchen to see what his mother had cooked
for the last time.  but it was already served, and the dish stood
on the hearth.  then he said, mother, what is there to eat to-day.
see for yourself, said his mother.  so thumbling jumped on to the
hearth, and peeped into the dish, but as he stretched his neck
in too far the steam from the food caught hold of him, and carried
him up the chimney.  he rode about in the air on the steam for a
while, until at length he sank down to the ground again

Creating a mapping from each unique character in the text to a unique number

In [6]:
#Storing all the unique characters present in the text

vocabulary = sorted(list(set(text)))

#Creating dictionaries to map each character to an index
char_to_indices = dict((c,i) for i, c in enumerate(vocabulary))

indices_to_char = dict((i,c) for i, c in enumerate(vocabulary))

print(vocabulary)

['\n', ' ', "'", ',', '-', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Preprocessing the data

In [7]:
max_length = 100
steps = 5
sentences = []
next_chars = []

for i in range(0, len(text) - max_length, steps):
    sentences.append(text[i : i + max_length])
    next_chars.append(text[i + max_length])
    
    
    
    
#Hot encoding each character into a boolean vector
X = np.zeros((len(sentences), max_length, len(vocabulary)), dtype = np.bool)
y = np.zeros((len(sentences), len(vocabulary)), dtype = np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_indices[char]] = 1
        
        
    y[i, char_to_indices[next_chars[i]]] = 1
    

Building the LSTM Network

In [8]:
model = Sequential()
model.add(LSTM(128, input_shape = (max_length, len(vocabulary))))
model.add(Dense(len(vocabulary)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr = 0.01)

model.compile(loss = 'categorical_crossentropy', optimizer = optimizer)

In [9]:
def sample_index(preds, temperature = 1.0):
    preds = np.array(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds/np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [10]:
def on_epoch_end(epoch, logs):
    print()
    print('------Generated text after epoch: %d' %epoch)
    start_index = random.randint(0, len(text) - max_length - 1)
    
    for diversity in[0.2, 0.5, 1.0, 1.2]:
        print('------diversity:', diversity)
        
        generated = ' '
        sentence = text[start_index: start_index + max_length]
        
        generated += sentence
        print('-----Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(400):
            x_pred = np.zeros((1, max_length, len(vocabulary)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_to_indices[char]] = 1
                
            preds = model.predict(x_pred, verbose = 0)[0]
            next_index = sample_index(preds, diversity)
            next_char = indices_to_char[next_index]
            
            generated += next_char
            
            sentence = sentence[1 :] + next_char
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
            
            
        print()    
        
        
        
        
print_callback = LambdaCallback(on_epoch_end = on_epoch_end)        
    

Save the model

In [11]:
filepath = "weigths.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')

In [12]:
reduce_alpha = ReduceLROnPlateau(monitor ='loss', factor = 0.2, 
                              patience = 1, min_lr = 0.001) 
callbacks = [print_callback, checkpoint, reduce_alpha] 

Training the LSTM model

In [13]:
model.fit(X, y, batch_size = 128, epochs = 10, callbacks = callbacks)

Epoch 1/10
1658/1658 [==============================] - 10s 6ms/step - loss: 3.2021

------Generated text after epoch: 0
------diversity: 0.2
-----Generating with seed: "dispatching the last with all his might, then hopped
nimbly upon it, and flew down with it through t"
 dispatching the last with all his might, then hopped
nimbly upon it, and flew down with it through th t t  a ia         ha   atet a et t        t   s  t  h th  i t  e t aa   aatat t t  tht   h   t  t  t a    t a t   t e  a t aeh t   a  a  t h  t t al t  h  t    th  a    tet  tat a  t ahe      h he      aatt   t n t t t   thh t  he  tht   t  a  t   t ah  o   a  h     tt  tt ist    t  i i  taa  t  th     as ttt  a   ha s   t   t te   a  t oh  e  tt  at ho    t i l tt   t  ee          i      t   t 
------diversity: 0.5
-----Generating with seed: "dispatching the last with all his might, then hopped
nimbly upon it, and flew down with it through t"
 dispatching the last with all his might, then hopped
nimbly upon it, and f

him, for he saw all they did secretly, without th onl ws iag  klilgo nwad isyacple tth plv litn apw he,
h stlnpnj youth, hafy ,re dih thmy andaye
thucheplgid bic jwaibs
and dets rodee  hont ag lebule bhe  seli  id teaed avaicy of.veed ouathl  aseew as oabeig iap  aatg lit tpr sitweel fht qiie  the the u hat hm poir ithecjonrakas icmhu the ee biet mepsthe coe dicges nithehe hase roing
-ang himr,noi g wok it gt cele sound ajwikf iust o saed ghit 

Epoch 00006: loss improved from 2.31508 to 2.15998, saving model to weigths.hdf5
Epoch 7/10
1658/1658 [==============================] - 9s 5ms/step - loss: 2.0345

------Generated text after epoch: 6
------diversity: 0.2
-----Generating with seed: "ht him and drove him out of the house.

the little tailor journeyed on and came to a great forest, a"
 ht him and drove him out of the house.

the little tailor journeyed on and came to a great forest, and he he the sall the sat he lit  he pe soud he sas he core the sas he fore he he sat he seal the

Generating text

In [18]:
def generate_text(length, diversity): 
    # Get random starting text
    start_index = random.randint(0, len(text) - max_length - 1) 
    generated = '' 
    sentence = text[start_index: start_index + max_length] 
    generated += sentence 
    for i in range(length): 
            x_pred = np.zeros((1, max_length, len(vocabulary))) 
            for t, char in enumerate(sentence): 
                x_pred[0, t, char_to_indices[char]] = 1.
  
            preds = model.predict(x_pred, verbose = 0)[0] 
            next_index = sample_index(preds, diversity) 
            next_char = indices_to_char[next_index] 
  
            generated += next_char 
            sentence = sentence[1:] + next_char 
    return generated 
  
print(generate_text(500, 0.2)) 





le tailor was outside in the wide world, and he traveled
about, and went to a master in his craft, but masthen the lat ho saill thum lith  ous he lit the sat the lat to he lit the sat he sat the lat to and he lit the saile wit wow the lat to he call the sat to he cor he sat hums ng he  wot and he fat the lat to wat to and there the said and he lat the lat the lat to he call the saile woud and he lat the saile woud and hams he cor he stoll the fatle the lat to sat the lat to and cave the said and ham the lit the lat to and there the wat hum lit the said and haved wow the litt and was and be ben
